# Inputs changes

We are here using the Inputs code provided on dm4bem book. Using a specific weather file and a start and end date, this code allows us to get :
- the total solar irradiance
- the solar radiation absorbed by the outdoor wall
- the solar radiation absorbed by the indoor wall
- the solar radiation absorbed by the glass

### Value for a week in february

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dm4bem

start_date = '02-01 12:00:00'
end_date = '02-07 18:00:00'

start_date = '2000-' + start_date
end_date = '2000-' + end_date
print(f'{start_date} \tstart date')
print(f'{end_date} \tend date')

# 1. Weather: outdoor temperature and solar radiation
filename = 'FRA_AR_Lyon-Bron.AP.074800_TMYx.2004-2018.epw'
[data, meta] = dm4bem.read_epw(filename, coerce_year=None)
weather = data[["temp_air", "dir_n_rad", "dif_h_rad"]]
del data

weather.index = weather.index.map(lambda t: t.replace(year=2000))
weather = weather.loc[start_date:end_date]

# Temperature sources
To = weather['temp_air']

# total solar irradiance
wall_out = pd.read_csv('walls_out.csv')  # Donnees mur exterieur
w0 = wall_out[wall_out['ID'] == 'w0']

surface_orientation = {'slope': w0['β'].values[0],
                       'azimuth': w0['γ'].values[0],
                       'latitude': 45}

rad_surf = dm4bem.sol_rad_tilt_surf(
    weather, surface_orientation, w0['albedo'].values[0])

Etot = rad_surf.sum(axis=1)  # Total solar irradiance

2000-02-01 12:00:00 	start date
2000-02-07 18:00:00 	end date


In order to get the maximum total solar irradiance between the two dates given :

In [2]:
Etotmax = max(Etot)          # Maximum of the total solar irradiance between both dates
print(f'Total solar irradiance in february : {Etotmax} W/m²')

Total solar irradiance in february : 444.4654797634998 W/m²


In [3]:
# Flow rate source
# solar radiation absorbed by the outdoor surface of the wall
phio = w0['α1'].values[0] * w0['Area'].values[0] * Etot
# window glass properties
α_gSW = 0.38    # short wave absortivity: reflective blue glass
τ_gSW = 0.30    # short wave transmitance: reflective blue glass
S_g = 9         # m², surface area of glass


We can also get the maximum solar radiation absorbed by the outdoor surface of the wall :

In [4]:
phiomax=max(phio)
print(f'Solar radiation absorbed by the outdoor surface of the wall in february : {phiomax} W/m²')

Solar radiation absorbed by the outdoor surface of the wall in february : 6000.283976807247 W/m²


In [5]:
# solar radiation absorbed by the indoor surface of the wall
Φi = τ_gSW * w0['α0'].values[0] * S_g * Etot

Then the maximum solar radiation absorbed by the indoor surface of the wall :

In [6]:
Φimax=max(Φi)
print(f'Solar radiation absorbed by the indoor surface of the wall in february : {Φimax} W/m²')

Solar radiation absorbed by the indoor surface of the wall in february : 300.0141988403623 W/m²


In [7]:
# solar radiation absorbed by the glass
Φa = α_gSW * S_g * Etot

And finally the same thing with the maximum solar radiation absorbed by the glass :

In [8]:
Φamax=max(Φa)
print(f'Solar radiation absorbed by the glass in february : {Φamax} W/m²')

Solar radiation absorbed by the glass in february : 1520.0719407911693 W/m²


In [9]:
# 2. Schedule
# indoor air temperature set-point
Ti_sp = pd.Series(20, index=To.index)

Ti_day, Ti_night = 20, 16

Ti_sp = pd.Series(
    [Ti_day if 6 <= hour <= 22 else Ti_night for hour in To.index.hour],
    index=To.index)

# auxiliary (internal) sources
Qa = 0 * np.ones(weather.shape[0])

# 3. Input Data set
input_data_set_february = pd.DataFrame({'To': To, 'Ti_sp': Ti_sp,
                               'Φo': phio, 'Φi': Φi, 'Qa': Qa, 'Φa': Φa,
                               'Etot': Etot})
input_data_set_february.to_csv('input_data_set_february.csv')

### Value for a week in August

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dm4bem

start_date = '08-01 12:00:00'
end_date = '08-07 18:00:00'

start_date = '2000-' + start_date
end_date = '2000-' + end_date
print(f'{start_date} \tstart date')
print(f'{end_date} \tend date')

# 1. Weather: outdoor temperature and solar radiation
filename = 'FRA_AR_Lyon-Bron.AP.074800_TMYx.2004-2018.epw'
[data, meta] = dm4bem.read_epw(filename, coerce_year=None)
weather = data[["temp_air", "dir_n_rad", "dif_h_rad"]]
del data

weather.index = weather.index.map(lambda t: t.replace(year=2000))
weather = weather.loc[start_date:end_date]

# Temperature sources
To = weather['temp_air']

# total solar irradiance
wall_out = pd.read_csv('walls_out.csv')  # Donnees mur exterieur
w0a = wall_out[wall_out['ID'] == 'w0']

surface_orientationa = {'slope': w0['β'].values[0],
                       'azimuth': w0['γ'].values[0],
                       'latitude': 45}

rad_surfa = dm4bem.sol_rad_tilt_surf(
    weather, surface_orientation, w0['albedo'].values[0])

Etota = rad_surfa.sum(axis=1)  # Total solar irradiance
Etotmaxa = max(Etota)          # Maximum of the total solar irradiance between both dates
print(f'Total solar irradiance in august : {Etotmax} W/m²')

# Flow rate source

# solar radiation absorbed by the outdoor surface of the wall
phioa = w0a['α1'].values[0] * w0a['Area'].values[0] * Etota
# window glass properties
α_gSW = 0.38    # short wave absortivity: reflective blue glass
τ_gSW = 0.30    # short wave transmitance: reflective blue glass
S_g = 9         # m², surface area of glass
phiomaxa=max(phioa)
print(f'Solar radiation absorbed by the outdoor surface of the wall in august : {phiomaxa} W/m²')

# solar radiation absorbed by the indoor surface of the wall
Φia = τ_gSW * w0a['α0'].values[0] * S_g * Etota
Φimaxa=max(Φia)
print(f'Solar radiation absorbed by the indoor surface of the wall in august : {Φimaxa} W/m²')

# solar radiation absorbed by the glass
Φaa = α_gSW * S_g * Etota
Φamaxa=max(Φaa)
print(f'Solar radiation absorbed by the glass in august : {Φamaxa} W/m²')

# 2. Schedule
# indoor air temperature set-point
Ti_sp = pd.Series(20, index=To.index)

Ti_day, Ti_night = 20, 16

Ti_sp = pd.Series(
    [Ti_day if 6 <= hour <= 22 else Ti_night for hour in To.index.hour],
    index=To.index)

# auxiliary (internal) sources
Qa = 0 * np.ones(weather.shape[0])

# 3. Input Data set
input_data_set_august = pd.DataFrame({'To': To, 'Ti_sp': Ti_sp,
                               'Φo': phioa, 'Φi': Φia, 'Qa': Qa, 'Φa': Φaa,
                               'Etot': Etota})
input_data_set_august.to_csv('input_data_set_august.csv')

2000-08-01 12:00:00 	start date
2000-08-07 18:00:00 	end date
Total solar irradiance in august : 444.4654797634998 W/m²
Solar radiation absorbed by the outdoor surface of the wall in august : 8421.583297183148 W/m²
Solar radiation absorbed by the indoor surface of the wall in august : 421.0791648591573 W/m²
Solar radiation absorbed by the glass in august : 2133.4677686197306 W/m²


### Comparison between february and august values

In [11]:
Mod1={'Results in february (W/m²)':Etotmax,'Results in august (W/m²)':Etotmaxa}
Mod2={'Results in february (W/m²)':phiomax,'Results in august (W/m²)':phiomaxa}
Mod3={'Results in february (W/m²)':Φimax,'Results in august (W/m²)':Φimaxa}
Mod4={'Results in february (W/m²)':Φamax,'Results in august (W/m²)':Φamaxa}
pd.DataFrame.from_dict({'Maximum total solar irradiance':Mod1,
                        'Maximum solar radiation absorbed by the outdoor surface of the wall':Mod2,
                        'Maximum solar radiation absorbed by the indoor surface of the wall':Mod3,
                        'Maximum solar radiation absorbed by the glass':Mod4,},orient='index')

,Results in february (W/m²),Results in august (W/m²)
Maximum total solar irradiance,444.465480,623.820985
Maximum solar radiation absorbed by the outdoor surface of the wall,6000.283977,8421.583297
Maximum solar radiation absorbed by the indoor surface of the wall,300.014199,421.079165
Maximum solar radiation absorbed by the glass,1520.071941,2133.467769


As expected, the maximum total solar irradiance is higher in august as in february. It seems logical as the sun is stronger in the summer in France. As a result, the maximum solar radiation absorbed by the different materials (outdoor and indoor surface of the wall and the glass) is higher in august than in february. It is about 1.5 times higher in august.